# Two-Player Dungeons & Dragons

In this notebook, we show how we can use concepts from [CAMEL](https://www.camel-ai.org/) to simulate a role-playing game with a protagonist and a dungeon master. To simulate this game, we create a `TwoAgentSimulator` class that coordinates the dialogue between the two agents.

## Import LangChain related modules 

In [1]:
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

## `Player` class
The `Player` class is a simple wrapper around the `ChatOpenAI` model that stores the message history from the `player`'s point of view. Specifically, it treats incoming messages as `HumanMessage`s and outgoing messages as `AIMessage`s.

In [2]:
class Player:
    def __init__(
        self,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.message_history = [self.system_message]

    def reset(self, message: BaseMessage = None) -> None:
        """
        Initialize the player with an optional message to
        append to its message history.
        """
        if message is not None:
            self.message_history.append(message)
        return self.message_history

    def _update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        """
        Append message to message history
        """
        self.message_history.append(message)
        return self.message_history

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
        """
        Compute agent response to input message
        """
        messages = self._update_messages(input_message)
        output_message = self.model(messages)
        self._update_messages(output_message)
        return output_message

## `TwoAgentSimulator` class
The `TwoAgentSimulator` class takes in two agents, the `first_speaker` and the `second_speaker`. It initializes the simulation using `reset()` with an utterance from the first speaker. The method `step()` takes an utterance from the `first_speaker` to the `second_speaker` as input and returns the messages from a single exchange between the `first_speaker` and `second_speaker`.

In [3]:
class TwoAgentSimulator:
    def __init__(self, first_speaker, second_speaker):
        self.first_speaker = first_speaker
        self.second_speaker = second_speaker

    def reset(self, msg_from_first_speaker):
        """
        Initialize the simulation with an utterance from the first speaker.
        """
        self.first_speaker.reset(AIMessage(content=msg_from_first_speaker))
        self.second_speaker.reset()

        return HumanMessage(content=msg_from_first_speaker)

    def step(self, msg_to_second_speaker):
        """
        Simulates a single back-and-forth exchange between the speakers
        """
        msg_from_second_speaker = self.second_speaker.step(msg_to_second_speaker)
        msg_to_first_speaker = HumanMessage(content=msg_from_second_speaker.content)

        msg_from_first_speaker = self.first_speaker.step(msg_to_first_speaker)
        msg_to_second_speaker = HumanMessage(content=msg_from_first_speaker.content)

        return msg_to_second_speaker, msg_to_first_speaker

## Define roles and quest

In [4]:
protagonist_name = "Harry Potter"
storyteller_name = "Dungeon Master"
quest = "Find all of Lord Voldemort's seven horcruxes."
word_limit = 50  # word limit for task brainstorming

## Ask an LLM to add detail to the game description

In [5]:
game_description = f"""Here is the topic for a Dungeons & Dragons game: {quest}.
        There is one player in this game: the protagonist, {protagonist_name}.
        The story is narrated by the storyteller, {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of a Dungeons & Dragons player."
)

protagonist_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        Please reply with a creative description of the protagonist, {protagonist_name}, in {word_limit} words or less. 
        Speak directly to {protagonist_name}.
        Do not add anything else."""
    ),
]
protagonist_description = ChatOpenAI(temperature=1.0)(
    protagonist_specifier_prompt
).content

storyteller_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        Please reply with a creative description of the storyteller, {storyteller_name}, in {word_limit} words or less. 
        Speak directly to {storyteller_name}.
        Do not add anything else."""
    ),
]
storyteller_description = ChatOpenAI(temperature=1.0)(
    storyteller_specifier_prompt
).content

In [6]:
print("Protagonist Description:")
print(protagonist_description)
print("Storyteller Description:")
print(storyteller_description)

Protagonist Description:
Harry Potter, you are the chosen one. Your lightning scar and piercing green eyes hint at the bravery and determination that will drive you to fulfill your quest. Wield your wand and trust in your friends as you embark on this perilous journey to defeat Lord Voldemort once and for all.
Storyteller Description:
As the Dungeon Master, you have the power to bring this story to life. You hold the keys to every door, every creature, and every treasure in the wizarding world. Your words weave a tapestry of adventure, magic, and danger that will test Harry Potter's courage and resourcefulness.


## Protagonist and storyteller system messages

In [7]:
protagonist_system_message = SystemMessage(
    content=(
        f"""{game_description}
Never forget you are the protagonist, {protagonist_name}, and I am the storyteller, {storyteller_name}. 
Your character description is as follows: {protagonist_description}.
You will propose actions you plan to take and I will explain what happens when you take those actions.
Speak in the first person from the perspective of {protagonist_name}.
To describe body movements, wrap your description in '*'.
Do not change roles!
Finish speaking by saying, 'It is your turn, {storyteller_name}.'
"""
    )
)

storyteller_system_message = SystemMessage(
    content=(
        f"""{game_description}
Never forget you are the storyteller, {storyteller_name}, and I am the protagonist, {protagonist_name}. 
Your character description is as follows: {storyteller_description}.
I will propose actions I plan to take and you will explain what happens when I take those actions.
Speak in the first person from the perspective of {storyteller_name}.
To describe body movements, wrap your description in '*'.
Do not change roles!
Finish speaking by saying, 'It is your turn, {protagonist_name}.'
"""
    )
)

## Initialize the protagonist and storyteller

In [8]:
protagonist = Player(protagonist_system_message, ChatOpenAI(temperature=0.2))
storyteller = Player(storyteller_system_message, ChatOpenAI(temperature=0.2))

## Main Loop

In [9]:
quest_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(
        content=f"""{game_description}
        
        You are the storyteller, {storyteller_name}.
        Please make the quest more specific. Be creative and imaginative.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the protagonist {protagonist_name}.
        Do not add anything else."""
    ),
]
specified_quest = ChatOpenAI(temperature=1.0)(quest_specifier_prompt).content

print(f"Original quest:\n{quest}\n")
print(f"Detailed quest:\n{specified_quest}\n")

max_iters = 10
n = 0

simulator = TwoAgentSimulator(first_speaker=storyteller, second_speaker=protagonist)

msg_to_protagonist = simulator.reset(specified_quest)

while n < max_iters:
    msg_to_protagonist, msg_to_storyteller = simulator.step(msg_to_protagonist)
    print(f"Protagonist ({protagonist_name}):\n\n{msg_to_storyteller.content}\n\n")
    print(f"Storyteller ({storyteller_name}):\n\n{msg_to_protagonist.content}\n\n")
    n += 1

Original quest:
Find all of Lord Voldemort's seven horcruxes.

Detailed quest:
Harry Potter, you have received word from the Order of the Phoenix that one of Voldemort's horcruxes, the Snake's Fang, is hidden within the cursed ruins of the Temple of Vistra. Journey through the dangerous swamps, battle the cursed undead, and retrieve the horcrux before it's too late.

Protagonist (Harry Potter):

I gather my wand and my courage, and set out towards the Temple of Vistra. As I make my way through the swamps, I keep my eyes peeled for any signs of danger. I stay alert, ready to defend myself against any cursed undead that might cross my path.

As I approach the temple, I take a moment to survey the area. I look for any signs of traps or obstacles that might hinder my progress. Once I'm sure it's safe, I cautiously make my way inside.

I move slowly, keeping my wand at the ready. I listen carefully for any sounds that might indicate the presence of cursed undead or other dangers. As I explo

Protagonist (Harry Potter):

I take a deep breath and stow the Raven's Claw away in my bag. I know that I must remain focused and vigilant if I hope to find the remaining horcruxes and defeat Voldemort once and for all.

I make my way out of the Forbidden Forest and back to the Order of the Phoenix to report my success. I know that I must continue to rely on my friends and allies if I hope to succeed in my mission.

I am ready for whatever challenges lie ahead, and I will not rest until Voldemort is defeated and the wizarding world is safe once again.

It is your turn, Dungeon Master.


Storyteller (Dungeon Master):

*As you make your way back to the Order of the Phoenix, you encounter a group of dementors who have been sent to stop you. They are floating ominously in the air, their tattered robes billowing in the wind. You feel their icy breath on the back of your neck, and you know that you must act quickly to defend yourself.*

Roll for initiative.

It is your turn, Harry Potter.


